In [ ]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime as dt
from bs4 import BeautifulSoup as BS
from time import sleep
from random import randint
import regex as re
from babel.numbers import format_currency

In [ ]:
#bring in gdp excel worksheet
gdp = pd.read_excel('../data/gdp.xlsx',header = [3], nrows = 3219)
gdp.head()

In [ ]:
#rename unamed column
gdp = gdp.rename(columns = {'Unnamed: 0':'County'})
gdp.head()

In [ ]:
#pull out columns for particular counties
counties = gdp.loc[(gdp['County']=='Davidson')|(gdp['County']=='Fulton')|(gdp['County']=='Travis')]
counties

In [ ]:
#locate proper fulton county
fulton = counties.iloc[1].to_frame().transpose().reset_index(drop = True)
fulton

In [ ]:
#locate proper davidson county
davidson = counties.iloc[9].to_frame().transpose().reset_index(drop=True)
davidson

In [ ]:
#locate proper travis county
travis = counties.iloc[10].to_frame().transpose().reset_index(drop=True)
travis

In [ ]:
#add city column
fulton['city'] = ['Atlanta']
fulton

In [ ]:
#add city column
davidson['city'] = ['Nashville']
davidson

In [ ]:
#add city column
travis['city'] = ['Austin']
travis

In [ ]:
#concatenating all counties
counties_gdp = pd.concat([fulton,davidson,travis])
counties_gdp

In [ ]:
#setting columns
counties_gdp = counties_gdp.loc[:,['County','city',2018,2019, 2020,2021, '2019.1','2020.1', '2021.2']]
counties_gdp

In [ ]:
#gross gdp DF
gross = counties_gdp[['city',2018,2019, 2020,2021]]
gross

In [ ]:
#percent change DF
perc_change = counties_gdp[['city', '2019.1','2020.1', '2021.2']]
perc_change

In [ ]:
#percent change DF renaming
perc_change = perc_change.rename(columns = {'2019.1':'2019','2020.1':'2020', '2021.2':'2021'})
perc_change

In [ ]:
#first melt for gdp value
gross = pd.melt(gross, id_vars = ['city'],var_name = 'year',value_name = 'gross_gdp')
gross

In [ ]:
#formatting gross gdp column to currency
gross['gross_gdp'] = gross['gross_gdp'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))
gross

In [ ]:
#first melt for gdp % change
perc_change = pd.melt(perc_change, id_vars = ['city'],var_name = 'year',value_name = 'gdp_perc_change')
perc_change

In [ ]:
gross.to_csv('../data/cleaned/economic/gross_gdp.csv', index = False)

In [ ]:
perc_change.to_csv('../data/cleaned/economic/gdp_perc_change.csv', index = False)

In [ ]:
price_parity = pd.read_csv('../data/price_parity.csv', nrows = 9)
price_parity

In [ ]:
price_parity[['rpp','desc','service']] = price_parity['Description'].str.split(': ', expand = True)
price_parity

In [ ]:
price_parity['service'].astype(str)

In [ ]:
#replacing None value with Goods
price_parity = price_parity.replace(to_replace ={ None: 'Goods'})
price_parity

In [ ]:
price_parity[['city','1','2','3','4','5']] = price_parity['GeoName'].str.split('-',expand = True)
price_parity

In [ ]:
price_parity = price_parity.loc[:,['city','service','2017','2018','2019','2020']]
price_parity 

In [ ]:
price_parity.to_csv('../data/cleaned/economic/price_parity.csv', index = False)